In [17]:
import pandas as pd
import numpy as np
import torch
import os
import umap

from tqdm import tqdm

In [18]:
DATA_FOLDER = "./data/"
CHECKPOINT_ID = "TEST"
DIR = os.path.join(DATA_FOLDER, CHECKPOINT_ID)

In [19]:
# UMAP settings:

N_NEIGHBORS = 3
MIN_DIST = 0.1
METRIC = 'cosine'
SEMI_SUPERVISED = False
SEMI_SUPERVISED_N = 10

In [20]:
def get_top_n_indices(df, column_name, n):
    return np.argsort(-df[column_name].values)[:n]

In [21]:
def get_bottom_n_indices(df, column_name, n):
    return np.argsort(df[column_name].values)[:n]

In [22]:
embeddings = np.load(os.path.join(DIR, "y_embeddings.npy"))
metadata = pd.read_csv(os.path.join(DIR, "metadata.csv"), index_col=0)

In [23]:
embeddings

array([[0.26739013, 0.287149  , 0.3533957 , ..., 0.9722748 , 0.42294902,
        0.4622926 ],
       [0.5191779 , 0.00640219, 0.37415892, ..., 0.4497561 , 0.8127194 ,
        0.22906065],
       [0.41835755, 0.8100909 , 0.9690676 , ..., 0.13887334, 0.7410607 ,
        0.8147367 ],
       ...,
       [0.14864844, 0.9890428 , 0.6018168 , ..., 0.65867513, 0.94109213,
        0.2263087 ],
       [0.10550368, 0.1704374 , 0.60778856, ..., 0.61049473, 0.9952317 ,
        0.08921719],
       [0.8005367 , 0.97020966, 0.8839034 , ..., 0.46284622, 0.14257395,
        0.47934908]], dtype=float32)

In [24]:
metadata

,audio_file,x_b,x_d,y_hat_b,y_hat_d,brightness_diff,depth_diff,p_drive,p_hpf,p_bias,p_output_db,p_hat_drive,p_hat_hpf,p_hat_bias,p_hat_output_db
id,,,,,,,,,,,,,,,
0,./audio0.wav,0.491584,0.366129,0.179557,0.043799,-0.312027,-0.322329,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
1,./audio1.wav,0.491584,0.366129,0.471837,0.252225,-0.019747,-0.113904,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
2,./audio2.wav,0.491584,0.366129,0.329368,0.900148,-0.162216,0.534020,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
3,./audio3.wav,0.491584,0.366129,0.205323,0.487679,-0.286260,0.121551,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
4,./audio4.wav,0.491584,0.366129,0.626251,0.029368,0.134667,-0.336761,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,./audio995.wav,0.491584,0.366129,0.798466,0.889531,0.306882,0.523403,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
996,./audio996.wav,0.491584,0.366129,0.763310,0.865694,0.271727,0.499565,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
997,./audio997.wav,0.491584,0.366129,0.219154,0.087598,-0.272429,-0.278530,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0


In [ ]:
def label_top_n_features(df, n, tolerance = 1_000):
    found_indices = False
    duplicates = []
    iterations = 0
    while not found_indices:
        top_bright = get_top_n_indices(df, 'brightness_diff', n)
        bottom_bright = get_bottom_n_indices(df, 'brightness_diff', n)
        
        top_depth = get_top_n_indices(df, 'depth_diff', n)
        bottom_depth = get_bottom_n_indices(df, 'depth_diff', n)
        
        iterations += 1
        
        if iterations >= tolerance:
            print(f"iterations surpassed tolerance of {tolerance}, ")
        

In [25]:
top_bright = get_top_n_indices(metadata, 'brightness_diff', 10)

In [15]:
metadata.iloc[top_bright]

,audio_file,x_b,x_d,y_hat_b,y_hat_d,brightness_diff,depth_diff,p_drive,p_hpf,p_bias,p_output_db,p_hat_drive,p_hat_hpf,p_hat_bias,p_hat_output_db
id,,,,,,,,,,,,,,,
725,./audio725.wav,0.491584,0.366129,0.999098,0.522078,0.507515,0.155949,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
527,./audio527.wav,0.491584,0.366129,0.997719,0.129203,0.506135,-0.236926,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
822,./audio822.wav,0.491584,0.366129,0.996991,0.289403,0.505408,-0.076725,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
879,./audio879.wav,0.491584,0.366129,0.996721,0.794545,0.505137,0.428416,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
606,./audio606.wav,0.491584,0.366129,0.996666,0.816140,0.505082,0.450011,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
556,./audio556.wav,0.491584,0.366129,0.995936,0.466975,0.504353,0.100846,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
454,./audio454.wav,0.491584,0.366129,0.995696,0.867752,0.504112,0.501623,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
840,./audio840.wav,0.491584,0.366129,0.995420,0.351875,0.503836,-0.014253,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
243,./audio243.wav,0.491584,0.366129,0.993055,0.057027,0.501471,-0.309102,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0


In [39]:
indices = [3,5,7]

data_slice = metadata[~metadata.index.isin(indices)]
np_slice = np.argsort(data_slice.reset_index()['id'].values)[:10]

data_slice.iloc[np_slice]

,audio_file,x_b,x_d,y_hat_b,y_hat_d,brightness_diff,depth_diff,p_drive,p_hpf,p_bias,p_output_db,p_hat_drive,p_hat_hpf,p_hat_bias,p_hat_output_db
id,,,,,,,,,,,,,,,
0,./audio0.wav,0.491584,0.366129,0.179557,0.043799,-0.312027,-0.322329,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
1,./audio1.wav,0.491584,0.366129,0.471837,0.252225,-0.019747,-0.113904,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
2,./audio2.wav,0.491584,0.366129,0.329368,0.900148,-0.162216,0.534020,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
4,./audio4.wav,0.491584,0.366129,0.626251,0.029368,0.134667,-0.336761,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
6,./audio6.wav,0.491584,0.366129,0.941832,0.295444,0.450248,-0.070684,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
8,./audio8.wav,0.491584,0.366129,0.948458,0.091993,0.456875,-0.274136,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
9,./audio9.wav,0.491584,0.366129,0.168726,0.376773,-0.322858,0.010644,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
10,./audio10.wav,0.491584,0.366129,0.269548,0.705820,-0.222035,0.339692,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
11,./audio11.wav,0.491584,0.366129,0.114210,0.904980,-0.377374,0.538852,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
